In [1]:
import autorootcwd

In [11]:
from rich import print
from rich.markdown import Markdown
import pandas as pd
import json
import openai
import re
from umap.umap_ import UMAP
import plotly.express as px
from tqdm import tqdm
from dataclasses import dataclass
from typing import List

In [12]:
with open('results/code_FE_method_extraction/gpt-3.5-turbo-1106/v2/answers.json', 'r') as file:
    json_data = json.load(file)

In [15]:
print(len(json_data))

methods_list = []
for record in json_data:
    methods_list += record["methods"]

print(len(methods_list))

300

1396

In [19]:
data = [item["method"] + " <br>" + item["execution"] for item in methods_list]
data[0]

'Data type conversion <br>Converted data types of certain columns to optimize memory usage and improve performance.'

In [20]:
client = openai.Client()

def get_embedding(text, model="text-embedding-ada-002"):
   # text = text.replace("\n", " ")
   response = client.embeddings.create(input = [text], model=model)
   return response.data[0].embedding

embeddings = [get_embedding(text) for text in tqdm(data)]

  0%|          | 0/1396 [00:00<?, ?it/s]

100%|██████████| 1396/1396 [07:27<00:00,  3.12it/s]


In [21]:
reducer = UMAP()
embeddings_umap = reducer.fit_transform(embeddings)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [22]:
# Convert reduced embeddings into a DataFrame for easier plotting
df_embeddings_umap = pd.DataFrame(embeddings_umap, columns=['x', 'y'])

# Create an interactive scatter plot using Plotly
fig = px.scatter(df_embeddings_umap, x='x', y='y')

# Update the hovertemplate to only show 'text'
fig.update_traces(hovertemplate='%{hovertext}', hovertext=data)

# set size of the plot
fig.update_layout(
    autosize=False,
    width=1400,
    height=1400,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
)

# Display the plot
fig.show()